In [1]:
import matplotlib.pyplot as plt
import time
import torch
from torch import logsumexp
from torch.distributions.normal import Normal
import numpy as np
from utils import *
from plots import *
from objectives_old import *

In [4]:
## training parameters
iterations = 40000
NUM_SAMPLES = 20
NUM_SAMPLES_SNR = 100
LEARNING_RATE = 1e-3
## model parameters
p_mu = 0.0
p_sigma2 = 1.0
log_Z = np.log(np.sqrt((2*np.pi)))

ests = ['IWAE', 'IWAE-DReG', 'RWS', 'RWS-DReG', 'STL']
PATH = 'IS-'

In [5]:
ELBOs = []
ESSs = []
SNRs = []
Gvars = []

for est in ests:
    q_mu = torch.tensor([3.0], requires_grad=True)
    q_sigma = torch.tensor([2.0], requires_grad=True) 
    optimizer = torch.optim.SGD([q_mu, q_sigma], lr=LEARNING_RATE)
    print('======= start training by %s ========\n' % est)
    if est == 'IWAE':
        ELBO, Mu, Sigma, Grad_mu, Grad_sigma, ESS = iwae(NUM_SAMPLES, q_mu, q_sigma, p_mu, p_sigma2, iterations, optimizer)
    elif est == 'IWAE-DReG':
        EUBO, ELBO, Mu, Sigma, Grad_mu, Grad_sigma, ESS = dreg(NUM_SAMPLES, q_mu, q_sigma, p_mu, p_sigma2, iterations, optimizer, alpha=0)
    elif est == 'RWS':
        EUBO, ELBO, Mu, Sigma, Grad_mu, Grad_sigma, ESS = rws(NUM_SAMPLES, q_mu, q_sigma, p_mu, p_sigma2, iterations, optimizer)
    elif est == 'RWS-DReG':
        EUBO, ELBO, Mu, Sigma, Grad_mu, Grad_sigma, ESS = dreg(NUM_SAMPLES, q_mu, q_sigma, p_mu, p_sigma2, iterations, optimizer, alpha=1)
    else:
        EUBO, ELBO, Mu, Sigma, Grad_mu, Grad_sigma, ESS = dreg(NUM_SAMPLES, q_mu, q_sigma, p_mu, p_sigma2, iterations, optimizer, alpha=0.5)
        
#     EUBOsmo, ELBOsmo = exp_average(EUBO, ELBO, iterations, beta1=0.9, beta2=0.99)   
    eg_mu, eg2_mu, var_mu, mu_snr = SNR(np.array(Grad_mu), iterations, beta1=0.99, beta2=0.99)    
    eg_sigma, eg2_sigma, var_sigma, sigma_snr = SNR(np.array(Grad_sigma), iterations, beta1=0.99, beta2=0.99)    
    
    ELBOs.append(ELBO)
    ESSs.append(ESS)
    Gvars.append((var_mu + var_sigma) / 2)
    SNRs.append((mu_snr + sigma_snr) / 2)
    print('======= end training by %s ========\n' % est)

======= start training by IWAE ========

iteration:0, ELBO:-0.407, ESS:1.973 (0s)
iteration:1000, ELBO:1.384, ESS:6.731 (0s)
iteration:2000, ELBO:1.404, ESS:7.652 (0s)
iteration:3000, ELBO:1.173, ESS:6.576 (0s)
iteration:4000, ELBO:0.397, ESS:4.073 (1s)
iteration:5000, ELBO:0.365, ESS:4.544 (1s)
iteration:6000, ELBO:0.723, ESS:5.811 (1s)
iteration:7000, ELBO:1.416, ESS:10.513 (1s)
iteration:8000, ELBO:0.855, ESS:6.362 (1s)
iteration:9000, ELBO:0.463, ESS:4.064 (0s)
iteration:10000, ELBO:0.982, ESS:6.385 (0s)
iteration:11000, ELBO:0.736, ESS:5.487 (0s)
iteration:12000, ELBO:0.215, ESS:5.012 (0s)
iteration:13000, ELBO:0.592, ESS:4.809 (0s)
iteration:14000, ELBO:0.785, ESS:5.321 (0s)
iteration:15000, ELBO:1.308, ESS:9.717 (0s)
iteration:16000, ELBO:0.866, ESS:7.025 (0s)
iteration:17000, ELBO:0.459, ESS:6.090 (1s)
iteration:18000, ELBO:0.329, ESS:4.143 (1s)
iteration:19000, ELBO:0.909, ESS:7.765 (1s)
iteration:20000, ELBO:1.336, ESS:10.550 (1s)
iteration:21000, ELBO:0.908, ESS:7.646 (1s)
i

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
def plot_results_multiple(ELBOs, ESSs, SNRs, num_samples, ests):
    colors = ['blue', 'red', 'orange', 'black', 'green']
    fig = plt.figure(figsize=(40,20))
    plt.tight_layout()
    axes = fig.subplots(3, 3, sharex=True)
    axes[1,0].set_yscale('log')
    axes[1,1].set_yscale('log')
    axes[1,2].set_yscale('log')
    for i, est in enumerate(ests):
        if est == 'IWAE' or est == 'IWAE-DReG':
            axes[0,0].plot(ELBOs[i], c=colors[i], label="ELBO " + est)
            axes[1,0].plot(SNRs[i], c=colors[i], label='SNR ' + est)
#             axes[1,0].plot(Gvars[i], c=colors[i], label='Grad var ' + est)
            ess_ratio = np.array(ESSs[i]) / num_samples
            ave_ess = np.reshape(ess_ratio, (-1, 10)).mean(-1)
            N = ave_ess.shape[0]
            axes[2,0].plot(np.arange(N) * 10, ave_ess, '-o', c=colors[i], label='ESS ' + est)
        elif est == 'RWS' or est == 'RWS-DReG':
            axes[0,1].plot(ELBOs[i], c=colors[i], label="ELBO " + est)
            axes[1,1].plot(SNRs[i], c=colors[i], label='SNR ' + est)
#             axes[1,1].plot(Gvars[i], c=colors[i], label='Grad var ' + est)
            ess_ratio = np.array(ESSs[i]) / num_samples
            ave_ess = np.reshape(ess_ratio, (-1, 10)).mean(-1)
            N = ave_ess.shape[0]
            axes[2,1].plot(np.arange(N) * 10, ave_ess, '-o', c=colors[i], label='ESS ' + est)
        else:   
            axes[0,2].plot(ELBOs[i], c=colors[i], label="ELBO " + est)
            axes[1,2].plot(SNRs[i], c=colors[i], label='SNR ' + est)
#             axes[1,2].plot(Gvars[i], c=colors[i], label='Grad var ' + est)
            ess_ratio = np.array(ESSs[i]) / num_samples
            ave_ess = np.reshape(ess_ratio, (-1, 10)).mean(-1)
            N = ave_ess.shape[0]
            axes[2,2].plot(np.arange(N) * 10, ave_ess, '-o', c=colors[i], label='ESS ' + est)
    axes[0,0].set_ylim([-10,2])   
    axes[0,1].set_ylim([-10,2]) 
    axes[0,2].set_ylim([-10,2])
    
    axes[1,0].set_ylim([1e-4,1e3])
    axes[1,1].set_ylim([1e-4,1e3])
    axes[1,2].set_ylim([1e-4,1e3])
    axes[1,0].set_yticks
    axes[2,0].set_ylim([0, 1])

    axes[0,0].legend()
    axes[1,0].legend()
    axes[2,0].legend()
    
    axes[0,1].legend()
    axes[1,1].legend()
    axes[2,1].legend()
    
    axes[0,2].legend()
    axes[1,2].legend()
    axes[2,2].legend()
    plt.savefig(PATH + 'training_results_%dsamples.svg' % num_samples)

In [ ]:
plot_results_multiple(ELBOs, ESSs, SNRs, num_samples, ests)